In [69]:
import pandas as pd
import json
import os
print("Combingin all data sources.........")

Combingin all data sources.........


In [70]:
all_conversations = []

In [71]:
# Survery form data
# Open your JSONL file properly
with open("data/processed/survey.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 44 conversations


In [72]:
# Empathetic Conversation data
empathetic_file = "data/raw/empatheticdialogues/train.csv"


if os.path.exists(empathetic_file):
    try:
        # First, let's check what columns exist
        print("📋 Reading CSV file...")
        # Try comma first
        try:
            df = pd.read_csv(empathetic_file, sep=',', encoding='utf-8', on_bad_lines='skip')
        except:
            # If comma fails, try tab
            print("⚠️ Comma separator failed, trying tab...")
            df = pd.read_csv(empathetic_file, sep='\t', encoding='utf-8', on_bad_lines='skip')
        
        # Print column names to debug
        print(f"📊 Columns found: {list(df.columns)}")
        print(f"📊 Total rows: {len(df)}")
        print(f"\n📄 First few rows:")
        print(df.head())
        
        # Clean column names (remove extra spaces)
        df.columns = df.columns.str.strip()
        
        # Check if conv_id exists
        if 'conv_id' not in df.columns:
            print(f"❌ 'conv_id' column not found. Available columns: {list(df.columns)}")
        else:
            # Group by conversation
            grouped = df.groupby('conv_id')
            
            for conv_id, group in grouped:
                # Sort by utterance index
                group = group.sort_values('utterance_idx')
                
                messages = []
                for _, row in group.iterrows():
                    role = 'user' if row['speaker_idx'] == 1 else 'assistant'
                    content = str(row['utterance']).strip()
                    if len(content) > 5:
                        messages.append({"role": role, "content": content})
                
                if len(messages) > 1:
                    all_conversations.append({"messages": messages})
            
            print(f"✅ Added {len(all_conversations)} conversations from EmpatheticDialogues")
    
    except Exception as e:
        print(f"⚠️ Error reading EmpatheticDialogues: {e}")
        import traceback
        print(traceback.format_exc())
else:
    print(f"⚠️ EmpatheticDialogues file not found at {empathetic_file}")

# Save to JSONL
if all_conversations:
    import json
    output_file = "data/processed/empathetic_conversations.jsonl"
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for conv in all_conversations:
            f.write(json.dumps(conv, ensure_ascii=False) + '\n')
    
    print(f"💾 Saved to {output_file}")

📋 Reading CSV file...
📊 Columns found: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags']
📊 Total rows: 76668

📄 First few rows:
        conv_id  utterance_idx      context  \
0  hit:0_conv:1              1  sentimental   
1  hit:0_conv:1              2  sentimental   
2  hit:0_conv:1              3  sentimental   
3  hit:0_conv:1              4  sentimental   
4  hit:0_conv:1              5  sentimental   

                                              prompt  speaker_idx  \
0  I remember going to the fireworks with my best...            1   
1  I remember going to the fireworks with my best...            0   
2  I remember going to the fireworks with my best...            1   
3  I remember going to the fireworks with my best...            0   
4  I remember going to the fireworks with my best...            1   

                                           utterance     selfeval tags  
0  I remember going to see the fireworks with my ... 

In [73]:
with open("data/processed/burmese.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 17878 conversations


In [74]:
# Save final Combined Data


output_file = "data/processed/combined3dataset.jsonl"

print(f"\n💾 Saving combined data...")
print(f"📊 Total conversations: {len(all_conversations)}")

with open(output_file, 'w', encoding='utf-8') as f:
    for conv in all_conversations:
        f.write(json.dumps(conv, ensure_ascii=False) + '\n')

print(f"✅ Saved to: {output_file}")


💾 Saving combined data...
📊 Total conversations: 17878
✅ Saved to: data/processed/combined3dataset.jsonl
